# Echo hiding watermarking

In [1]:
import librosa
import numpy as np
import scipy
from matplotlib import pyplot as plt

In [9]:
# read audio data
x, sr = librosa.load("sample.wav",sr=None)

# frame segmentation
sig_length = len(x)
frame_length = 4096
hop_length = 2048
overlap = frame_length - hop_length
Nframe = int( (sig_length-overlap) / hop_length)

# Hann window
win = scipy.signal.windows.hann(frame_length)

# message generation
if_repetitive_coding = True
if if_repetitive_coding:
    repeat = 3
    msg_len = int(Nframe/repeat)
else:
    repeat = 1
    msg_len = Nframe
        
msg = np.random.randint(2, size=msg_len)

In [6]:
# echo kernels
delta1 = 100
delta0 = 120

alpha = 0.2 #echo amplitude
delay = 4 # negative echo

prev1 = np.zeros(frame_length) #for positive single
prev2 = np.zeros(frame_length) #for pos/neg pair
prev3 = np.zeros(frame_length) #for back/forward

## watermark embedding

In [ ]:
embedded_x = np.copy(x)
for frame_idx in range(Nframe):
    start = librosa.frames_to_samples(frame_idx,hop_length=hop_length)
    frame = x[start:start+hop_length]
    
    if frame_idx//repeat < msg_len:
        msg_bit = msg[frame_idx//repeat]
        delta = delta1 if msg_bit==1 else delta0
    else:
        delta = 0
    
    echo_p = alpha * np.append(np.zeros(delta), frame[:frame_length-delta])
    echo_n = -alpha * np.append(np.zeros(delta+delay), frame[:frame_length-delta-delay])
    echo_f = alpha * np.append(frame[delta:], np.zeros(delta))
    
    echoed_frame1 = frame + echo_p
    echoed_frame2 = frame + echo_p + echo_n
    echoed_frame3 = frame + echo_p + echo_f
    
    # windowing and smoothing
    
    